In [1]:
import pickle
import pandas as pd

In [2]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [3]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [4]:
year = 2022
month = 2
taxi_type = 'yellow'
input_file = f"https://d37ci6vzurychx.cloudfront.net/trip-data/{taxi_type}_tripdata_{year:04d}-{month:02d}.parquet"
df = read_data(input_file)

In [6]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

### Q1 Notebook

In [10]:
# Q1 STDDEV of y_pred
y_pred.std()

5.28140357655334

### Q2 Preparing the output

In [11]:
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [19]:
import os
df['duration_prediction'] = y_pred
# Write duration_prediction and ride_id to a new dataframe
df_result = df[['ride_id', 'duration_prediction']]
output_file = f"output/{taxi_type}/{year:04d}-{month:02d}.parquet"
os.makedirs(os.path.dirname(output_file), exist_ok=True)
df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)